# Read Data

1) Preparing the data. 

In [1]:
import glob

In [2]:

def read_text_files(folder_path):
    file_paths = glob.glob(folder_path + '/*.txt')
    texts = []
    for file_path in file_paths:
        with open(file_path, 'r', encoding='utf-8') as file:
            text = file.read()
            texts.append(text)
    return texts



In [1]:
import os

def read_text_files(folder_path):
    texts = []
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.txt'):
                file_path = os.path.join(root, file)
                with open(file_path, 'r', encoding='utf-8') as file:
                    text = file.read()
                    texts.append(text)
    return texts


In [2]:

# gutenberg_folder_path = '/home/judal/Documentos/CURSOS_2023_I/BigData/tarea_1/Gutenberg_Text/Alcott, Louisa May'
gutenberg_folder_path = '/home/judal/Documentos/CURSOS_2023_I/BigData/tarea_1/Gutenberg_Text'
texts = read_text_files(gutenberg_folder_path)
# gutenberg_text = ' '.join(texts)

In [3]:
len(texts)

3383

# MapReduce Implementation

2) Writing map and reduce functions.

<!-- 3) Getting a better understanding of how mapper and reducer work. -->

In [4]:
import string
import re
import pandas as pd
import itertools

In [4]:
def mapper(chunk):
    word_count = {}
    words = chunk.split()
    for word in words:
        if word not in word_count:
            word_count[word] = 0
        word_count[word] += 1
    return word_count

In [4]:
def reducer(word_counts):
    final_word_count = {}
    for word_count in word_counts:
        for word, count in word_count.items():
            if word not in final_word_count:
                final_word_count[word] = 0
            final_word_count[word] += count
    return final_word_count

In [8]:
data = [
    "Lorem ipsum dolor sit amet",
    "consectetur adipiscing elit",
    "sed do eiusmod tempor incididunt ut labore et dolore magna aliqua"
]

In [5]:
def mapreduce(data, mapper, reducer):
    word_counts = []
    for text in data:
        word_count = mapper(text)
        word_counts.append(word_count)
    
    result = reducer(word_counts)
    return result


# 25%, 50% y 100%

In [6]:
n_ = len(texts)
p = 1
texts_split = texts[:round(n_*p)]

In [6]:
len(texts_split)

1692

In [19]:
%%time
result = mapreduce(texts_split,mapper,reducer)


CPU times: user 30.9 s, sys: 300 ms, total: 31.2 s
Wall time: 32.4 s


In [20]:
from itertools import islice
firts_100 = dict(islice(result.items(),1000))
firts_100

{'The': 438772,
 'Project': 129351,
 'Gutenberg': 34921,
 'eBook,': 2080,
 'A': 80238,
 'Survey': 76,
 'of': 3201853,
 'Russian': 4563,
 'Literature,': 86,
 'with': 887280,
 'Selections,': 2,
 'by': 460432,
 'Isabel': 361,
 'Florence': 1300,
 'Hapgood': 32,
 'This': 64158,
 'eBook': 8877,
 'is': 724959,
 'for': 698837,
 'the': 5723237,
 'use': 32502,
 'anyone': 10174,
 'anywhere': 5292,
 'at': 595442,
 'no': 249486,
 'cost': 7539,
 'and': 3252272,
 'almost': 41864,
 'restrictions': 3464,
 'whatsoever.': 3390,
 'You': 76471,
 'may': 123407,
 'copy': 15678,
 'it,': 85107,
 'give': 54601,
 'it': 711634,
 'away': 56933,
 'or': 402355,
 're-use': 3280,
 'under': 69005,
 'terms': 35188,
 'License': 13154,
 'included': 4895,
 'this': 396911,
 'online': 6591,
 'www.gutenberg.org': 2218,
 'Title:': 1678,
 'Selections': 33,
 'Author:': 1672,
 'Release': 1695,
 'Date:': 1980,
 'April': 1929,
 '3,': 820,
 '2007': 115,
 '[eBook': 288,
 '#20980]': 1,
 'Language:': 1669,
 'English': 16581,
 'Characte

## 25% 50% 100%


In [7]:
n_ = len(texts)
p = 1
texts_split = texts[:round(n_*p)]

In [8]:
len(texts_split)

3383

In [9]:
import threading

In [10]:
word_counts = []
final_word_counts = []
lock = threading.Lock()

In [11]:
word_counts

[]

In [12]:
def mapT(chunk,i,j,data):
    
    global word_counts
    
    subdata = data[i:j]

    for text in subdata:
        word_count = {}
        words = text.split()
        for word in words:
            if word not in word_count:
                word_count[word] = 0
            word_count[word] += 1
        with lock:
            word_counts.append(word_count)
    
    print("hello",chunk, "->",i," : ",j)

In [13]:
def reduceT(chunk,i,j,word_counts):
    
    global final_word_counts

    subdata = word_counts[i:j]

    # print(len(subdata))

    # for word_count in subdata:
    #     print(chunk,", ",type(word_count)) 


    final_word_count = {}
    for word_count in subdata:    
        for word, count  in word_count.items():
            if word not in final_word_count:
                final_word_count[word] = 0
            final_word_count[word] += count
    with lock:
        final_word_counts.append(final_word_count)
            
    print("hello",chunk, "->",i," : ",j)

In [14]:

def proccessingMap(data, mapT, num_threads):
    threads = []

    k = len(data) // num_threads
    
    for _ in range(num_threads):

        inicio = _ * k
        fin = (_ * k) + k if _ < num_threads - 1 else len(data)

        thread = threading.Thread(target=mapT,args=(_,inicio,fin,data))
        thread.start()
        threads.append(thread)

    for thread in threads:
        thread.join()

In [15]:
def proccessingReduce(word_counts, reduceT, num_threads):
    threads = []

    k = len(word_counts) // num_threads
    
    for _ in range(num_threads):

        inicio = _ * k
        fin = (_ * k) + k if _ < num_threads - 1 else len(word_counts)

        thread = threading.Thread(target=reduceT,args=(_,inicio,fin,word_counts))
        thread.start()
        threads.append(thread)

    for thread in threads:
        thread.join()


### Map Processing

In [16]:
%%time
num_threads = 8
proccessingMap(texts_split, mapT, num_threads)


hello 3 -> 1266  :  1688
hello 1 -> 422  :  844
hello 4 -> 1688  :  2110
hello 7 -> 2954  :  3383
hello 2 -> 844  :  1266
hello 6 -> 2532  :  2954
hello 5 -> 2110  :  2532
hello 0 -> 0  :  422
CPU times: user 44.1 s, sys: 1.1 s, total: 45.2 s
Wall time: 44.5 s


In [17]:
print(len(texts_split))
print(len(word_counts))

3383
3383


In [20]:
print(texts[0][:100])
# for word in word_counts:
#     print(type(word), end=", ")

The Project Gutenberg eBook, A Survey of Russian Literature, with
Selections, by Isabel Florence Hap


### Reduce Processing

In [18]:
%%time

num_threads = 8
proccessingReduce(word_counts, reduceT, num_threads)


hello 0 -> 0  :  422
hello 1 -> 422  :  844
hello 3 -> 1266  :  1688
hello 2 -> 844  :  1266
hello 6 -> 2532  :  2954
hello 7 -> 2954  :  3383
hello 5 -> 2110  :  2532
hello 4 -> 1688  :  2110
CPU times: user 13.8 s, sys: 235 ms, total: 14 s
Wall time: 13.8 s


In [19]:
len(final_word_counts)

8

In [20]:
%%time
result = reducer(final_word_counts)

CPU times: user 2.51 s, sys: 124 ms, total: 2.64 s
Wall time: 2.69 s


In [21]:
result

{'The': 913708,
 'Project': 255772,
 'Gutenberg': 69433,
 'eBook,': 4011,
 'A': 166521,
 'Survey': 127,
 'of': 6562332,
 'Russian': 8009,
 'Literature,': 159,
 'with': 1740167,
 'Selections,': 3,
 'by': 977322,
 'Isabel': 1658,
 'Florence': 2138,
 'Hapgood': 33,
 'This': 139641,
 'eBook': 17251,
 'is': 1477761,
 'for': 1441762,
 'the': 11588014,
 'use': 67235,
 'anyone': 21728,
 'anywhere': 10132,
 'at': 1164283,
 'no': 492807,
 'cost': 16201,
 'and': 6250891,
 'almost': 81049,
 'restrictions': 6843,
 'whatsoever.': 6742,
 'You': 159043,
 'may': 262558,
 'copy': 31482,
 'it,': 162527,
 'give': 122927,
 'it': 1393252,
 'away': 103519,
 'or': 807959,
 're-use': 6468,
 'under': 137590,
 'terms': 70348,
 'License': 25964,
 'included': 9297,
 'this': 815026,
 'online': 12991,
 'www.gutenberg.org': 4598,
 'Title:': 3296,
 'Selections': 49,
 'Author:': 3295,
 'Release': 3340,
 'Date:': 3913,
 'April': 3745,
 '3,': 1660,
 '2007': 219,
 '[eBook': 543,
 '#20980]': 1,
 'Language:': 3289,
 'Englis

In [22]:
len(result)

2924639

In [33]:
result['France']

11532

In [34]:
result['cat']

5235

In [37]:
result['Cakes']

80

In [40]:
result['War']

2354

In [39]:
result['valley']

6047

# WRITE FILE

In [31]:
import json

In [32]:
with open('./result.txt','w') as convert_file:
    convert_file.write(json.dumps(result))

## test


## test


## test


## test


In [53]:
import threading

def hello(chunk):
    print("Hello, World!",chunk)

if __name__ == "__main__":
    threads = []
    num_threads = 8

    for _ in range(num_threads):
        thread = threading.Thread(target=hello,args=(_,))
        thread.start()
        threads.append(thread)

    for thread in threads:
        thread.join()

Hello, World! 0
Hello, World! 1
Hello, World! 2
Hello, World! 3
Hello, World! 4
Hello, World! 5
Hello, World! 6
Hello, World! 7
